In [3]:

from datetime import date
import humanize

print(['date.{}'.format(fn) for fn in dir(date) if not fn.startswith('_')])
print(['humanize.{}'.format(fn) for fn in dir(humanize) if not fn.startswith('_')])

['date.ctime', 'date.day', 'date.fromisocalendar', 'date.fromisoformat', 'date.fromordinal', 'date.fromtimestamp', 'date.isocalendar', 'date.isoformat', 'date.isoweekday', 'date.max', 'date.min', 'date.month', 'date.replace', 'date.resolution', 'date.strftime', 'date.timetuple', 'date.today', 'date.toordinal', 'date.weekday', 'date.year']
['humanize.activate', 'humanize.apnumber', 'humanize.clamp', 'humanize.deactivate', 'humanize.filesize', 'humanize.fractional', 'humanize.i18n', 'humanize.importlib_metadata', 'humanize.intcomma', 'humanize.intword', 'humanize.naturaldate', 'humanize.naturalday', 'humanize.naturaldelta', 'humanize.naturalsize', 'humanize.naturaltime', 'humanize.number', 'humanize.ordinal', 'humanize.precisedelta', 'humanize.scientific', 'humanize.thousands_separator', 'humanize.time']


In [40]:

humanize.intcomma(humanize.apnumber(1_000))

'1,000'

In [43]:

humanize.intcomma(humanize.apnumber(9))

'nine'

In [72]:

# Return a natural representation of a timedelta or number of seconds.
from datetime import timedelta

humanize.precisedelta(timedelta(minutes=(140+30)/11.5))

'14 minutes and 46.96 seconds'

In [63]:

# Return a natural representation of a time in a resolution that makes sense.
from datetime import timedelta

humanize.naturaltime(timedelta(minutes=120))

'2 hours ago'

In [52]:

# Return a natural day.
# For date values that are tomorrow, today or yesterday compared to
# present day return representing string. Otherwise, return a string
# formatted according to `format`.
from datetime import timedelta

date_obj = date.today() - timedelta(days=1)
humanize.naturalday(date_obj)

'yesterday'

In [49]:

# Like `naturalday`, but append a year for dates more than ~five months away.
from datetime import timedelta

date_obj = date.today() + timedelta(days=6*30)
humanize.naturaldate(date_obj)

'Oct 06 2022'

In [76]:

print(['humanize.time.{}'.format(fn) for fn in dir(humanize.time) if not fn.startswith('_')])

['humanize.time.Enum', 'humanize.time.Unit', 'humanize.time.dt', 'humanize.time.intcomma', 'humanize.time.math', 'humanize.time.naturaldate', 'humanize.time.naturalday', 'humanize.time.naturaldelta', 'humanize.time.naturaltime', 'humanize.time.precisedelta', 'humanize.time.total_ordering']


In [77]:

humanize.time.total_ordering?

Signature: humanize.time.total_ordering(cls)
Docstring: Class decorator that fills in missing ordering methods
File:      c:\programdata\anaconda3\lib\functools.py
Type:      function


In [32]:

print(['humanize.filesize.{}'.format(fn) for fn in dir(humanize.filesize) if not fn.startswith('_')])

['humanize.filesize.naturalsize', 'humanize.filesize.suffixes']


In [31]:

print(['humanize.i18n.{}'.format(fn) for fn in dir(humanize.i18n) if not fn.startswith('_')])

['humanize.i18n.activate', 'humanize.i18n.deactivate', 'humanize.i18n.get_translation', 'humanize.i18n.gettext_module', 'humanize.i18n.local', 'humanize.i18n.os', 'humanize.i18n.thousands_separator']


In [37]:

print(['humanize.importlib_metadata.{}'.format(fn) for fn in dir(humanize.importlib_metadata) if not fn.startswith('_')])

['humanize.importlib_metadata.ConfigParser', 'humanize.importlib_metadata.Distribution', 'humanize.importlib_metadata.DistributionFinder', 'humanize.importlib_metadata.EntryPoint', 'humanize.importlib_metadata.FastPath', 'humanize.importlib_metadata.FileHash', 'humanize.importlib_metadata.MetaPathFinder', 'humanize.importlib_metadata.MetadataPathFinder', 'humanize.importlib_metadata.PackageNotFoundError', 'humanize.importlib_metadata.PackagePath', 'humanize.importlib_metadata.PathDistribution', 'humanize.importlib_metadata.Prepared', 'humanize.importlib_metadata.abc', 'humanize.importlib_metadata.collections', 'humanize.importlib_metadata.csv', 'humanize.importlib_metadata.distribution', 'humanize.importlib_metadata.distributions', 'humanize.importlib_metadata.email', 'humanize.importlib_metadata.entry_points', 'humanize.importlib_metadata.files', 'humanize.importlib_metadata.functools', 'humanize.importlib_metadata.import_module', 'humanize.importlib_metadata.io', 'humanize.importlib_

In [35]:

humanize.i18n.thousands_separator()

','

In [67]:

print(['humanize.number.{}'.format(fn) for fn in dir(humanize.number) if not fn.startswith('_')])

['humanize.number.Fraction', 'humanize.number.NS_', 'humanize.number.P_', 'humanize.number.apnumber', 'humanize.number.clamp', 'humanize.number.fractional', 'humanize.number.human_powers', 'humanize.number.intcomma', 'humanize.number.intword', 'humanize.number.math', 'humanize.number.ordinal', 'humanize.number.powers', 'humanize.number.re', 'humanize.number.scientific', 'humanize.number.thousands_separator']


In [69]:

humanize.number.scientific?

Signature: humanize.number.scientific(value, precision=2)
Docstring:
Return number in string scientific notation z.wq x 10ⁿ.

Examples:
    ```pycon
    >>> scientific(float(0.3))
    '3.00 x 10⁻¹'
    >>> scientific(int(500))
    '5.00 x 10²'
    >>> scientific(-1000)
    '1.00 x 10⁻³'
    >>> scientific(1000, 1)
    '1.0 x 10³'
    >>> scientific(1000, 3)
    '1.000 x 10³'
    >>> scientific("99")
    '9.90 x 10¹'
    >>> scientific("foo")
    'foo'
    >>> scientific(None) is None
    True

    ```

Args:
    value (int, float, str): Input number.
    precision (int): Number of decimal for first part of the number.

Returns:
    str: Number in scientific notation z.wq x 10ⁿ.
File:      c:\programdata\anaconda3\lib\site-packages\humanize\number.py
Type:      function


In [17]:

from datetime import datetime

# print(['datetime.{}'.format(fn) for fn in dir(datetime) if not fn.startswith('_')])
def how_long_its_been(from_date_str, to_date_str=None, suppress=[]):
    from_datetime_obj = datetime.strptime(from_date_str, '%B %d, %Y')
    if to_date_str is None:
        to_datetime_obj = datetime.now()
    else:
        to_datetime_obj = datetime.strptime(to_date_str, '%B %d, %Y')
    how_long_str = humanize.precisedelta(to_datetime_obj - from_datetime_obj, suppress=suppress,
                                         minimum_unit='days', format='%0.0f')
    
    return how_long_str

In [22]:

secs_float = 819.4461708068848
humanize.precisedelta(secs_float, minimum_unit='minutes', format='%0.0f')

'14 minutes'

In [18]:

starting_date = 'January 1, 2011'
ending_date = 'October 31, 2011'
how_long_its_been(starting_date, ending_date, suppress=['months'])

'303 days'

In [19]:

marriage_date = 'June 14, 1986'
how_long_its_been(marriage_date)

'35 years, 10 months and 1 days'

In [20]:

steves_birth_date = 'November 5, 1990'
how_long_its_been(steves_birth_date)

'31 years, 5 months and 8 days'

In [21]:

jays_birth_date = 'September 30, 1996'
how_long_its_been(jays_birth_date)

'25 years, 6 months and 12 days'

In [19]:

from datetime import timedelta

date.today() - timedelta(days=3*365)

datetime.date(2019, 4, 2)